In [1]:
import glob
import os, shutil, sys, time, tqdm
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF

import numpy as np

from PIL import Image
from PIL.ExifTags import TAGS

In [3]:
### 準備資料夾路徑

dirs = ['train', 'valid', 'test', 'source']  # source for temp process
classes = ['0', '1', '2', '3', 'pp']   # pp for temp process
for d in dirs:
    try:
        os.mkdir('datasets/{}'.format(d))
    except:
        pass
        
    for c in classes:
        try:
            os.mkdir('datasets/{}/{}'.format(d, c))
        except:
            pass

# try:
#     os.mkdir('datasets/source')
# except:
#     pass

try:
    os.mkdir('datasets/source_bak')
except:
    pass    

In [3]:
## 把檔案從 to_ocr_print_to_img/output 各資料夾中全部複製到  'datasets/source - bak'

data_source_path = 'C:\\develop\\annotation_tools\\to_ocr_print_to_img\\output\\*\\*\\*.jpg'
target_path = 'datasets/source_bak'
filelist = glob.glob(data_source_path)
print(len(filelist))

limit = None
for f in tqdm.tqdm(filelist[:limit]):
    shutil.copy(f, target_path)

29560


100%|██████████| 29560/29560 [00:16<00:00, 1773.33it/s]


In [4]:
### remove filename space and special character

target_path = 'datasets\\source_bak'
filelist = glob.glob(os.path.join(target_path, '*.jpg'))
print(len(filelist))

for f in tqdm.tqdm(filelist):
    fname = os.path.basename(f).split('.jpg')[0]
    replace_list = ['[', ']', ' ', '(', ')', '【', '】', '.', ',', '《', '》', '、', '。', '「', '」', '『', '』', '-']
    for x in replace_list:
        fname = fname.replace(x, '')
    os.rename(f, os.path.join(target_path, fname+'.jpg'))


29560


100%|██████████| 29560/29560 [00:03<00:00, 8260.55it/s]


In [ ]:
### 說明 先將上一步的圖片用 尺寸 排序，就幾乎可以將同一類方向排一起，手動搬到對應的資料夾
### 剩下(留下來的)的圖片，就是要執行下面的程式 

In [5]:
## !!!執行後圖片可能會被轉方向!!!
# remove exif 

target_path = 'datasets/source_bak'
filelist = glob.glob(os.path.join(target_path, '*.jpg'))
print(len(filelist))

for f in tqdm.tqdm(filelist):
	img = Image.open(f)
	exif = img._getexif()

	if exif is not None:
		for (tag, value) in exif.items():
			key = TAGS.get(tag, tag)
			# print(key , ' = ' , str(value))
		
		data = list(img.getdata())
		image_without_exif = Image.new(img.mode, img.size)
		image_without_exif.putdata(data)
		image_without_exif.save(f)


29560


100%|██████████| 29560/29560 [00:30<00:00, 979.93it/s] 


# 上面的完成後，用眼睛確認實際方向並手動移動檔案到對應的 source/0,1,2,3 裡面
# 全部用下方程式旋轉到正向，可以反覆執行(0不會執行)

In [2]:
filelist = glob.glob('datasets/source/1/*.jpg')    # 原圖為向右，用程式左轉90度轉正
for demo_file in tqdm.tqdm(filelist):
    img = Image.open(demo_file)
    img=img.rotate(90, expand = True)  # 90 left -90 right -180 upside-down
    img.save(demo_file)
    
filelist = glob.glob('datasets/source/2/*.jpg')   # 原圖為向下，用程式轉180度轉正
for demo_file in tqdm.tqdm(filelist):
    img = Image.open(demo_file)
    img=img.rotate(-180, expand = True)  # 90 left -90 right -180 upside-down
    img.save(demo_file)
    
filelist = glob.glob('datasets/source/3/*.jpg')   # 原圖為向左，用程式右轉90度轉正
for demo_file in tqdm.tqdm(filelist):
    img = Image.open(demo_file)
    img=img.rotate(-90, expand = True)  # 90 left -90 right -180 upside-down
    img.save(demo_file)
    


0it [00:00, ?it/s]
100%|██████████| 3866/3866 [03:01<00:00, 21.27it/s] 


In [3]:
# remove exif again
def clearExifInfo(filelist):
    for f in tqdm.tqdm(filelist):
        img = Image.open(f)
        exif = img._getexif()

        if exif is not None:
            for (tag, value) in exif.items():
                key = TAGS.get(tag, tag)
                print(key , ' = ' , str(value))
            
            data = list(img.getdata())
            image_without_exif = Image.new(img.mode, img.size)
            image_without_exif.putdata(data)
            image_without_exif.save(f)

filelist = glob.glob('datasets/source/*/*.jpg') 
clearExifInfo(filelist)


100%|██████████| 29560/29560 [00:05<00:00, 5583.78it/s]


In [4]:
# 檢查好沒問題再搬 , (然後搬移到 pp 資料夾中，再做後續流程)
for i in [0, 1, 2, 3]:
    filelist = glob.glob('datasets/source/{}/*.jpg'.format(i))
    for f in tqdm.tqdm(filelist):
        try:
            shutil.move(f, 'datasets/source/pp')
        except:
            pass

100%|██████████| 25690/25690 [00:20<00:00, 1232.80it/s]
0it [00:00, ?it/s]
100%|██████████| 3866/3866 [00:07<00:00, 542.62it/s]


# 搬到 source/pp 之後，再次清除 image exif (exten info) 

In [5]:
### 搬到 source/pp 之後，清除 image exif (exten info)

def clearExifInfo(filelist):
    for f in tqdm.tqdm(filelist):
        img = Image.open(f)
        exif = img._getexif()

        if exif is not None:
            for (tag, value) in exif.items():
                key = TAGS.get(tag, tag)
                print(key + ' = ' + str(value))
            
            data = list(img.getdata())
            image_without_exif = Image.new(img.mode, img.size)
            image_without_exif.putdata(data)
            image_without_exif.save(f)
 
filelist = glob.glob('datasets/source/pp/*.jpg')
clearExifInfo(filelist)

# filelist = glob.glob('datasets/source/others/*.jpg')
# clearExifInfo(filelist)

100%|██████████| 29560/29560 [00:03<00:00, 7451.75it/s]


In [4]:
### 照設定比例拆分，搬到 train valid test 的 pp 裡面
filelist = glob.glob('datasets/source/pp/*.jpg')
filelist = filelist + glob.glob('datasets/source/others/*.jpg')   # 這裡把 others 也加進來
print(filelist[100:110])
new_filelist = [x[:-12] for x in filelist]   # _600_001.jpg -> 共12個字元
print(new_filelist[100:110])
distinct_filelist = list(set(new_filelist))
print(len(distinct_filelist))
print(distinct_filelist[10:20])

# 同一份文件風格類似，所以用文件來分割，不用圖片(頁面)來分割，否則模型可能會學到文件風格，而不是頁面方向特徵
train_list, test_list = train_test_split(distinct_filelist, test_size=0.05, shuffle=True)
train, valid = train_test_split(train_list, test_size=0.2, shuffle=True)
print('train', len(train))
print('valid', len(valid))
print('test_list', len(test_list))



['datasets/source/pp\\04OLAP_300_026.jpg', 'datasets/source/pp\\04OLAP_300_027.jpg', 'datasets/source/pp\\04OLAP_300_028.jpg', 'datasets/source/pp\\04OLAP_300_029.jpg', 'datasets/source/pp\\04OLAP_300_030.jpg', 'datasets/source/pp\\04OLAP_300_031.jpg', 'datasets/source/pp\\04OLAP_300_032.jpg', 'datasets/source/pp\\04OLAP_300_033.jpg', 'datasets/source/pp\\04OLAP_300_034.jpg', 'datasets/source/pp\\04OLAP_300_035.jpg']
['datasets/source/pp\\04OLAP', 'datasets/source/pp\\04OLAP', 'datasets/source/pp\\04OLAP', 'datasets/source/pp\\04OLAP', 'datasets/source/pp\\04OLAP', 'datasets/source/pp\\04OLAP', 'datasets/source/pp\\04OLAP', 'datasets/source/pp\\04OLAP', 'datasets/source/pp\\04OLAP', 'datasets/source/pp\\04OLAP']
264
['datasets/source/pp\\fb231104095216', 'datasets/source/pp\\combinepdf', 'datasets/source/pp\\今周刊20230803第1389期羅智先的新霸略', 'datasets/source/pp\\hw1', 'datasets/source/pp\\數位時代202308第351期新台劇闖全球', 'datasets/source/pp\\agreement', 'datasets/source/pp\\人工智慧之相關法規國際發展趨勢與因應委託研究計畫結案報

In [5]:
thislist = []
for f in tqdm.tqdm(train):
    thislist = glob.glob(f+'_*_*.jpg')
    for ff in thislist:
        shutil.copy(ff, 'datasets/train/pp')

thislist = []    
for f in tqdm.tqdm(valid):
    thislist = glob.glob(f+'_*_*.jpg')
    for ff in thislist:
        shutil.copy(ff, 'datasets/valid/pp')

thislist = []
for f in tqdm.tqdm(test_list):
    thislist = glob.glob(f+'_*_*.jpg')
    for ff in thislist:
        shutil.copy(ff, 'datasets/test/pp')


100%|██████████| 14/14 [00:05<00:00,  2.63it/s]


# about 20 mins

In [6]:
# 要先搬到 train valid test 資料夾裡面的 0 1 2 3 (旋轉方向)之後，才能做 img_padding & img_resize
# 大約要 20 分鐘

dirs = ['train', 'valid', 'test']  # source for temp process

for d in dirs:
    filelist = glob.glob('datasets/{}/pp/*.jpg'.format(d))
    i = 0
    for demo_file in tqdm.tqdm(filelist):
        try:
            img = ''
            img = Image.open(demo_file)
            # 改成每張圖片只放一個方向 其餘方向不放
            if i%4 == 0:
                img.save('datasets/{}/0/{}'.format(d, demo_file.split('\\')[-1]))     # 0 for original
            elif i%4 == 1:
                img=img.rotate(-90, expand = True)
                img.save('datasets/{}/1/{}'.format(d, demo_file.split('\\')[-1]))
            elif i%4 == 2:
                img=img.rotate(-180, expand = True)
                img.save('datasets/{}/2/{}'.format(d, demo_file.split('\\')[-1]))
            elif i%4 == 3:
                img=img.rotate(90, expand = True)
                img.save('datasets/{}/3/{}'.format(d, demo_file.split('\\')[-1]))
            else:
                pass
            i+=1
            # img1=img1.rotate(-90, expand = True)  # -90 right
            # img1.save('datasets/{}/1/{}'.format(d, demo_file.split('\\')[-1]))     # 1 for right
            # img2=img.copy()
            # img2=img2.rotate(-180, expand = True)  # -180 upside-down
            # img2.save('datasets/{}/2/{}'.format(d, demo_file.split('\\')[-1]))     # 2 for upside-down
            # img3=img.copy()
            # img3=img3.rotate(90, expand = True)  # 90 left
            # img3.save('datasets/{}/3/{}'.format(d, demo_file.split('\\')[-1]))     # 3 for left
        except:
            print('error', demo_file)
            pass
                 
    shutil.rmtree('datasets/{}/pp'.format(d))
    print('rmtree {} pp'.format(d))

  7%|▋         | 1425/21107 [00:12<01:18, 249.17it/s]

error datasets/train/pp\101年度國民消費意識及消費行為調查_300_026.jpg
error datasets/train/pp\101年度國民消費意識及消費行為調查_300_055.jpg


  9%|▉         | 1921/21107 [00:18<02:39, 120.16it/s]

error datasets/train/pp\107年行政院網路購物消費者意識及行為調查_200_019.jpg


 48%|████▊     | 10193/21107 [01:48<02:17, 79.50it/s] 

...

rmtree train pp


100%|██████████| 5789/5789 [01:00<00:00, 95.98it/s] 


rmtree valid pp


100%|██████████| 2692/2692 [00:27<00:00, 98.49it/s] 


rmtree test pp


In [4]:
# 再做一次 remove exif

def clearExifInfo(filelist):
    for f in tqdm.tqdm(filelist):
        img = Image.open(f)
        exif = img._getexif()

        if exif is not None:
            for (tag, value) in exif.items():
                key = TAGS.get(tag, tag)
                print(key + ' = ' + str(value))
            
            data = list(img.getdata())
            image_without_exif = Image.new(img.mode, img.size)
            image_without_exif.putdata(data)
            image_without_exif.save(f)
 
filelist = glob.glob('datasets/train/*/*.jpg')
clearExifInfo(filelist)

filelist = glob.glob('datasets/valid/*/*.jpg')
clearExifInfo(filelist)

filelist = glob.glob('datasets/test/*/*.jpg')
clearExifInfo(filelist)

  0%|          | 0/3788 [00:00<?, ?it/s]

100%|██████████| 250/250 [00:00<00:00, 1264.27it/s]


In [9]:
def cv2imread(img_path): 
    '''
    處理 windows cv2 讀取中文檔名的問題
    import cv2
    '''
    return cv2.imdecode(np.fromfile(img_path, dtype=np.uint8), cv2.IMREAD_UNCHANGED)   # 保留色彩

def img_padding(img):
    h, w = img.shape[:2]
    if h > w:
        img = cv2.copyMakeBorder(img, 0, 0, 0, h-w, cv2.BORDER_CONSTANT, value=(128,128,128))   # top, bottom, left, right
    elif w > h:
        img = cv2.copyMakeBorder(img, 0, w-h, 0, 0, cv2.BORDER_CONSTANT, value=(128,128,128))
    return img


def img_preprocess(img_path, d, t, name):
    # five_crop
    n = name.split('\\')[-1][:-4] # 取檔名 去掉 .jpg
    img = Image.open(img_path)
    cs = TF.five_crop(img, 400)
    sz = 224
    for i, c in enumerate(cs):
        c = TF.resize(c, (sz,sz))
        c.save('datasets/{}/{}/{}_{}.jpg'.format(d, t, n, i))
  
    
def img_resize_only(img_path, d, t, name): # add padding
    '''
    img : img path str
    path : train, valid, test
    name : img path str
    '''
    n = name.split('\\')[-1][:-4] # 取檔名 去掉 .jpg
    img = cv2imread(img_path)
    img = img_padding(img)
    img = Image.fromarray(img)    
    sz = 224
    ss = TF.resize(img, (sz,sz))
    ss.save('datasets/{}/{}/{}.jpg'.format(d, t, n))   # 覆蓋原檔案     

print('ok')    

ok


# 這就真的非常久了

In [10]:
dirs = ['train', 'valid', 'test']
types = ['0', '1', '2', '3']
count = 0
for d in dirs:
    if d == 'train':  # only train do five_crop
        for t in types:
            filelist = glob.glob('datasets/' + d + '/' + t + '/*.jpg')
            for f in tqdm.tqdm(filelist):
                # five_crop
                img_preprocess(f, d, t, f)
                img_resize_only(f, d, t, f)
                count += 1
    else:
        for t in types:
            filelist = glob.glob('datasets/' + d + '/' + t + '/*.jpg')
            for f in tqdm.tqdm(filelist):
                img_resize_only(f, d, t, f)
                count += 1

print(count)


100%|██████████| 673/673 [00:12<00:00, 53.85it/s]

29582


In [11]:
print('done')

done


In [13]:
# 計算全部多少張圖片
count = 0
dirs = ['train', 'valid', 'test']
for d in dirs:
    filelist = glob.glob('datasets/' + d + '/*/*.jpg')
    count += len(filelist)
print(count)

135087


In [ ]:
# epub to pdf